In [1]:
from pyspark.sql import SparkSession
import os
import sys
import pandas as pd
import pyspark.sql.functions as F
import numpy as np

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
spark=SparkSession.builder \
.master('local[*]') \
.appName('first_spark') \
.config("spark.executor.memory", "8g") \
.config("spark.driver.memory", "8g") \
.config("spark.executor.cores", "8") \
.getOrCreate() #создаем сессию спарк или обращаемся к существующей указываем машину локальную
    

In [5]:
prev=spark.read.csv('linkage/blockunited.csv')

In [6]:
prev

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]

In [7]:
prev.show(5)

+-------+-------+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|    _c0|    _c1|              _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-------+-------+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|   id_1|   id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|37291.0|53113.0|0.833333333333333|           ?|         1.0|           ?|    1.0|     1|     1|     1|      0|    True|
|39086.0|47614.0|                1|           ?|         1.0|           ?|    1.0|     1|     1|     1|      1|    True|
|70031.0|70237.0|                1|           ?|         1.0|           ?|    1.0|     1|     1|     1|      1|    True|
|84795.0|97439.0|                1|           ?|         1.0|           ?|    1.0|     1|     1|     1|      1|    True|
+-------+-------+---------------

In [9]:
parsed=(
    spark.read # Обращаемся к сессии на чтение
    .option('header',True) # Говорим что есть загаловки
    .option('nullValue','?')
    .option('inferSchema',True) #говорим чтобы формат данных сохранялся из ориг файла.Иначе бы все было строками(числа и тд)
    .csv('linkage/blockunited.csv') # Формат файла и путь до него
)

In [10]:
parsed.show(5)

+-------+-------+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|   id_1|   id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-------+-------+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291.0|53113.0|0.833333333333333|        NULL|         1.0|        NULL|    1.0|   1.0|   1.0|   1.0|    0.0|    true|
|39086.0|47614.0|              1.0|        NULL|         1.0|        NULL|    1.0|   1.0|   1.0|   1.0|    1.0|    true|
|70031.0|70237.0|              1.0|        NULL|         1.0|        NULL|    1.0|   1.0|   1.0|   1.0|    1.0|    true|
|84795.0|97439.0|              1.0|        NULL|         1.0|        NULL|    1.0|   1.0|   1.0|   1.0|    1.0|    true|
|36950.0|42116.0|              1.0|        NULL|         1.0|         1.0|    1.0|   1.0|   1.0|   1.0|    1.0|    true|
+-------+-------+---------------

In [11]:
parsed.printSchema()

root
 |-- id_1: double (nullable = true)
 |-- id_2: double (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: double (nullable = true)
 |-- cmp_bd: double (nullable = true)
 |-- cmp_bm: double (nullable = true)
 |-- cmp_by: double (nullable = true)
 |-- cmp_plz: double (nullable = true)
 |-- is_match: boolean (nullable = true)



In [12]:
parsed.count()

5749133

In [13]:
parsed = parsed.dropna(subset=['is_match'])

In [14]:
parsed.count()

5749132

In [15]:
from pyspark.sql.functions import col, when
parsed = parsed.withColumn("id_1", col("id_1").cast("int")) \
       .withColumn("id_2", col("id_2").cast("int")) \
       .withColumn("cmp_fname_c1", col("cmp_fname_c1").cast("double")) \
       .withColumn("cmp_fname_c2", col("cmp_fname_c2").cast("double")) \
       .withColumn("cmp_lname_c1", col("cmp_lname_c1").cast("double")) \
       .withColumn("cmp_lname_c2", col("cmp_lname_c2").cast("double")) \
       .withColumn("cmp_sex", col("cmp_sex").cast("int")) \
       .withColumn("cmp_bd", col("cmp_bd").cast("int")) \
       .withColumn("cmp_bm", col("cmp_bm").cast("int")) \
       .withColumn("cmp_by", col("cmp_by").cast("int")) \
       .withColumn("cmp_plz", col("cmp_plz").cast("int")) \
       .withColumn("is_match", col("is_match").cast("boolean"))

In [16]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [17]:
parsed.show(5)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        NULL|         1.0|        NULL|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        NULL|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
+-----+-----+-----------------+------------+------------+-------

In [18]:
parsed.count()

5749132

In [19]:
parsed.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

In [21]:
parsed.groupBy('is_match').count().orderBy(F.col('count').desc()).show(5)

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [22]:
parsed.agg(F.avg('cmp_sex'),F.stddev('cmp_sex')).show()

+-----------------+------------------+
|     avg(cmp_sex)|   stddev(cmp_sex)|
+-----------------+------------------+
|0.955001381078048|0.2073011111689783|
+-----------------+------------------+



In [23]:
parsed.createOrReplaceTempView('linkage') # Создаем временное представление нашего df с именнем linkage в котором будем выполнять SQL запросы

In [24]:
spark.sql('''
            SELECT
                is_match,
                COUNT(*) AS cnt
            FROM 
                linkage
            GROUP BY
                is_match
            ORDER BY
                cnt DESC
          ''').show(5)

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [25]:
spark.sql('''                      
            SELECT 
                is_match,
                COUNT(*) AS cnt
            FROM 
                linkage
            GROUP BY
                is_match
            ORDER BY
                cnt DESC
          ''').show(5)

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [26]:
parsed.describe().show()

+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|summary|              id_1|             id_2|       cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|           cmp_sex|             cmp_bd|             cmp_bm|             cmp_by|            cmp_plz|
+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|  count|           5749132|          5749132|            5748125|             103698|            5749132|               2464|           5749132|            5748337|            5748337|            5748337|            5736289|
|   mean| 33324.48559643438|66587.43558331935| 0.7129024704429269|  0.900017671890322|0.31562781

In [27]:
summary=parsed.describe()

In [28]:
summary.show()

+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|summary|              id_1|             id_2|       cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|           cmp_sex|             cmp_bd|             cmp_bm|             cmp_by|            cmp_plz|
+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|  count|           5749132|          5749132|            5748125|             103698|            5749132|               2464|           5749132|            5748337|            5748337|            5748337|            5736289|
|   mean| 33324.48559643438|66587.43558331935| 0.7129024704429269|  0.900017671890322|0.31562781

In [29]:
summary.select('summary','id_1','cmp_fname_c1','cmp_fname_c2','cmp_sex').show() #Выбираем определенные столбцы

+-------+------------------+-------------------+-------------------+------------------+
|summary|              id_1|       cmp_fname_c1|       cmp_fname_c2|           cmp_sex|
+-------+------------------+-------------------+-------------------+------------------+
|  count|           5749132|            5748125|             103698|           5749132|
|   mean| 33324.48559643438| 0.7129024704429269|  0.900017671890322| 0.955001381078048|
| stddev|23659.859374488096|0.38875835961628025|0.27131761057823345|0.2073011111689783|
|    min|                 1|                0.0|                0.0|                 0|
|    max|             99980|                1.0|                1.0|                 1|
+-------+------------------+-------------------+-------------------+------------------+



In [30]:
matches=parsed.filter(F.col('is_match')==True)
matches.show(5)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        NULL|         1.0|        NULL|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        NULL|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
+-----+-----+-----------------+------------+------------+-------

In [31]:
matches=parsed.where('is_match=true')
matches.show(5)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        NULL|         1.0|        NULL|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        NULL|         1.0|        NULL|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        NULL|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
+-----+-----+-----------------+------------+------------+-------

In [32]:
matches_summary=matches.describe()
matches_summary.show()

+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+
|summary|              id_1|             id_2|       cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|              cmp_bd|              cmp_bm|             cmp_by|            cmp_plz|
+-------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+
|  count|             20931|            20931|              20922|               1333|              20931|                475|              20931|               20925|               20925|              20925|              20902|
|   mean| 34575.72117911232|51259.95939037791| 0.9973163859635041| 0.989890032031817

In [33]:
misses=parsed.filter(F.col('is_match')==False)
misses_summary=misses.describe()
misses_summary.show()

+-------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+-----------------+------------------+--------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|       cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|           cmp_sex|             cmp_bd|           cmp_bm|            cmp_by|             cmp_plz|
+-------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+-----------------+------------------+--------------------+
|  count|           5728201|           5728201|           5727203|             102365|           5728201|               1989|           5728201|            5727412|          5727412|           5727412|             5715387|
|   mean|33319.913548075565| 66643.44259218557|0.7118634802167144| 0.8988473514090186| 0.313138011335959|0.1

Мы можем транспонировать фрейм данных, используя только функции, предоставляемые PySpark. Однако есть и другой способ выполнить эту задачу. PySpark по- зволяет конвертировать данные между фреймами Spark и pandas. Мы преобразуем рассматриваемые фреймы данных Spark в фреймы данных pandas, изменим их форму и преобразуем обратно в фреймы Spark. Благодаря небольшому размеру фреймов данных summary, match_summary I miss_summary мы можем делать это без опа- сений, хотя фреймы данных pandas целиком размещаются в памяти. Для аналогич- ных преобразований больших наборов данных в следующих главах мы будем ис- пользовать операции Spark.

Преобразование в формат pandas и обратно стало возможно благодаря проекту Apache Arrow, который обеспечивает эффективную передачу данных между про- цессами ЈѴМ и Python. Библиотека РуArrow была установлена как зависимость модуля Spark SQL, когда мы установили рyspark [sql] с помощью рір.

In [35]:
summary_pandas=summary.toPandas() #Преобразовываем в pandas dataframe

In [36]:
summary_pandas.head()

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
1,mean,33324.48559643438,66587.43558331935,0.7129024704429269,0.900017671890322,0.31562781930754885,0.31841283153174393,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
2,stddev,23659.859374488096,23620.48761326977,0.38875835961628025,0.27131761057823345,0.33423363396158345,0.36856706620066537,0.2073011111689783,0.4172297223846244,0.499875823677903,0.41609096298317494,0.07414914925420026
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [37]:
summary_pandas.shape

(5, 12)

Далее мы можем выполнить операцию транспонирования, чтобы поменять местами строки и столбцы, используя знакомые методы pandas:
summary_p = summary_p.set_index('summary').transpose().reset_index ()
42 | Глава 2
summary_p summary_p.rename (columns={'index':'field')) =
summary_p = summary_p.rename_axis (None, axis=1)
summary p.shape

In [38]:
summary_pandas = summary_pandas.set_index('summary') #Делаем колонку саммари индексом

In [39]:
summary_pandas.head()

,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
summary,,,,,,,,,,,
count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
mean,33324.48559643438,66587.43558331935,0.7129024704429269,0.900017671890322,0.31562781930754885,0.31841283153174393,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
stddev,23659.859374488096,23620.48761326977,0.38875835961628025,0.27131761057823345,0.33423363396158345,0.36856706620066537,0.2073011111689783,0.4172297223846244,0.499875823677903,0.41609096298317494,0.07414914925420026
min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [40]:
summary_pandas = summary_pandas.transpose()  #Меняем индексы на имена столбцов
summary_pandas.head()

summary,count,mean,stddev,min,max
id_1,5749132,33324.48559643438,23659.859374488096,1,99980
id_2,5749132,66587.43558331935,23620.48761326977,6,100000
cmp_fname_c1,5748125,0.7129024704429269,0.38875835961628025,0.0,1.0
cmp_fname_c2,103698,0.900017671890322,0.27131761057823345,0.0,1.0
cmp_lname_c1,5749132,0.31562781930754885,0.33423363396158345,0.0,1.0


In [41]:
summary_pandas = summary_pandas.reset_index() #Сбрасываем индексы(колонку summary),в итоге то что было в ней становится столбцом с именем индекс
                                              # Сама summary остается индексом
summary_pandas.head()

summary,index,count,mean,stddev,min,max
0,id_1,5749132,33324.48559643438,23659.859374488096,1,99980
1,id_2,5749132,66587.43558331935,23620.48761326977,6,100000
2,cmp_fname_c1,5748125,0.7129024704429269,0.38875835961628025,0.0,1.0
3,cmp_fname_c2,103698,0.900017671890322,0.27131761057823345,0.0,1.0
4,cmp_lname_c1,5749132,0.31562781930754885,0.33423363396158345,0.0,1.0


In [42]:
summary_pandas=summary_pandas.rename(columns={'index':'field'})
summary_pandas.head()

summary,field,count,mean,stddev,min,max
0,id_1,5749132,33324.48559643438,23659.859374488096,1,99980
1,id_2,5749132,66587.43558331935,23620.48761326977,6,100000
2,cmp_fname_c1,5748125,0.7129024704429269,0.38875835961628025,0.0,1.0
3,cmp_fname_c2,103698,0.900017671890322,0.27131761057823345,0.0,1.0
4,cmp_lname_c1,5749132,0.31562781930754885,0.33423363396158345,0.0,1.0


In [43]:
summary_pandas=summary_pandas.rename_axis(None, axis=1) #Каким то образом убираем первый столбец
summary_pandas.head()

,field,count,mean,stddev,min,max
0,id_1,5749132,33324.48559643438,23659.859374488096,1,99980
1,id_2,5749132,66587.43558331935,23620.48761326977,6,100000
2,cmp_fname_c1,5748125,0.7129024704429269,0.38875835961628025,0.0,1.0
3,cmp_fname_c2,103698,0.900017671890322,0.27131761057823345,0.0,1.0
4,cmp_lname_c1,5749132,0.31562781930754885,0.33423363396158345,0.0,1.0


In [44]:
summary_pandas.shape

(11, 6)

##### 2 способ

Метод melt в Pandas используется для преобразования данных из широкого формата в длинный формат. Это полезно, когда вам нужно сделать из данных с большим количеством столбцов данные с меньшим количеством столбцов, но большим количеством строк.

DataFrame.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

Параметры
id_vars: Указывает, какие столбцы должны оставаться неизменными (идентификаторы).

value_vars: Указывает, какие столбцы нужно трансформировать в строки (по умолчанию используются все столбцы, не указанные в id_vars).

var_name: Имя нового столбца, который будет содержать имена исходных столбцов.

value_name: Имя нового столбца, который будет содержать значения из исходных столбцов.

col_level: Если колонки являются MultiIndex, указывает, какой уровень использовать.

ignore_index: Если True, то старый индекс будет отброшен.

In [45]:
summary_pandas2=summary.toPandas()

In [47]:
summary_pandas2.head(6)

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
1,mean,33324.48559643438,66587.43558331935,0.7129024704429269,0.900017671890322,0.31562781930754885,0.31841283153174393,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
2,stddev,23659.859374488096,23620.48761326977,0.38875835961628025,0.27131761057823345,0.33423363396158345,0.36856706620066537,0.2073011111689783,0.4172297223846244,0.499875823677903,0.41609096298317494,0.07414914925420026
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [48]:
# Преобразуем DataFrame в формат с одним столбцом для имен колонок
melted_df = summary_pandas2.melt(id_vars=['summary'], var_name='field', value_name='value')
melted_df


,summary,field,value
0,count,id_1,5749132
1,mean,id_1,33324.48559643438
2,stddev,id_1,23659.859374488096
3,min,id_1,1
4,max,id_1,99980
5,count,id_2,5749132
6,mean,id_2,66587.43558331935
7,stddev,id_2,23620.48761326977
8,min,id_2,6
9,max,id_2,100000


In [49]:
# Сделаем pivot таблицу, чтобы значения 'summary' стали именами колонок
pivot_table = melted_df.pivot(index='field', columns='summary', values='value').reset_index()

# Выведем результат
display(pivot_table)

summary,field,count,max,mean,min,stddev
0,cmp_bd,5748337,1,0.22446526708507172,0,0.4172297223846244
1,cmp_bm,5748337,1,0.48885529849763504,0,0.499875823677903
2,cmp_by,5748337,1,0.2227485966810923,0,0.41609096298317494
3,cmp_fname_c1,5748125,1.0,0.7129024704429269,0.0,0.38875835961628025
4,cmp_fname_c2,103698,1.0,0.900017671890322,0.0,0.27131761057823345
5,cmp_lname_c1,5749132,1.0,0.31562781930754885,0.0,0.33423363396158345
6,cmp_lname_c2,2464,1.0,0.31841283153174393,0.0,0.36856706620066537
7,cmp_plz,5736289,1,0.00552866147434343,0,0.07414914925420026
8,cmp_sex,5749132,1,0.955001381078048,0,0.2073011111689783
9,id_1,5749132,99980,33324.48559643438,1,23659.859374488096


Мы успешно транспонировали фрейм данных summary_p в pandas. Преобразуем его в
Spark, используя метод createDataFrame B SparkSession:

In [50]:
summary_T=spark.createDataFrame(summary_pandas)

In [51]:
print(summary_T.dtypes)

[('field', 'string'), ('count', 'string'), ('mean', 'string'), ('stddev', 'string'), ('min', 'string'), ('max', 'string')]


In [52]:
summary_T.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [53]:
summary_T.show()

+------------+-------+-------------------+-------------------+---+------+
|       field|  count|               mean|             stddev|min|   max|
+------------+-------+-------------------+-------------------+---+------+
|        id_1|5749132|  33324.48559643438| 23659.859374488096|  1| 99980|
|        id_2|5749132|  66587.43558331935|  23620.48761326977|  6|100000|
|cmp_fname_c1|5748125| 0.7129024704429269|0.38875835961628025|0.0|   1.0|
|cmp_fname_c2| 103698|  0.900017671890322|0.27131761057823345|0.0|   1.0|
|cmp_lname_c1|5749132|0.31562781930754885|0.33423363396158345|0.0|   1.0|
|cmp_lname_c2|   2464|0.31841283153174393|0.36856706620066537|0.0|   1.0|
|     cmp_sex|5749132|  0.955001381078048| 0.2073011111689783|  0|     1|
|      cmp_bd|5748337|0.22446526708507172| 0.4172297223846244|  0|     1|
|      cmp_bm|5748337|0.48885529849763504|  0.499875823677903|  0|     1|
|      cmp_by|5748337| 0.2227485966810923|0.41609096298317494|  0|     1|
|     cmp_plz|5736289|0.00552866147434

In [54]:
summary_T.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [55]:
from pyspark.sql.types import DoubleType
for c in summary_T.columns:
    if c == 'field':
        continue
    else:
        summary_T=summary_T.withColumn(c,summary_T[c].cast(DoubleType()))
        
summary_T.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



withColumn(c, ...) создает или заменяет столбец c в DataFrame.

summaryT[c] обращается к текущему столбцу c.

.cast(DoubleType()) преобразует тип данных столбца в DoubleType.

In [72]:
def pivot_summary(desc):
    from pyspark.sql import DataFrame
    from pyspark.sql.types  import DoubleType
    # Конвертируем в пандас Датафрейм
    desc_pandas=desc.toPandas()
    #Транспонируем
    desc_pandas=desc_pandas.set_index('summary').transpose().reset_index()
    desc_pandas=desc_pandas.rename(columns={'index':'field'})
    desc_pandas=desc_pandas.rename_axis(None,axis=1)
    # Конвертируем в спарк Датафрейм
    desc_pandas=spark.createDataFrame(desc_pandas)
    #Меняем тип данных со стринг на дабл
    for c in desc_pandas.columns:
        if c == 'field':
            continue
        else:
            desc_pandas=desc_pandas.withColumn(c,desc_pandas[c].cast(DoubleType()))
    return desc_pandas

In [73]:
matches_summaryT=pivot_summary(matches_summary)
misses_summaryT=pivot_summary(misses_summary)

Делаем в SQL:

In [74]:
matches_summaryT.createOrReplaceTempView('matches_desc')
misses_summaryT.createOrReplaceTempView('misses_desc')

In [75]:
spark.sql('''
        SELECT
            ma.field,
            ma.count + mi.count AS total,
            ma.mean - mi.mean AS delta
        FROM
            matches_desc AS ma
        INNER JOIN
            misses_desc AS mi
        ON
            ma.field = mi.field
        WHERE
            ma.field NOT IN ('id_1', 'id_2')
        ORDER BY
            delta DESC,
            total DESC
          ''').show()

+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926268|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|  0.6838772482599227|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0|  0.2854529057467897|
|cmp_fname_c2| 103698.0| 0.09104268062279897|
|     cmp_sex|5749132.0|0.032408185250332844|
+------------+---------+--------------------+



Хороший признак имеет два свойства: у него существенно различаются значения для совпадений и несоответствий (поэтому разница между средними значениями будет большой), и он встречается в данных достаточно часто, так что мы можем надеяться, что он регулярно доступен для любой пары записей. По этим критериям cmp_fname_c2 нам не подходит, потому что состоит по большей части из пропусков, а разница в среднем значении для совпадений и несовпадений относительно мала 0,09 для оценки, которая колеблется от 0 до 1. Признак cmp_sex также не особенно полезен, потому что, хотя он доступен для любой пары записей, разница в средних значениях составляет всего 0,03.

Зато признаки cmp_plz и cmp_by просто превосходны. Они почти всегда встречаются
для любой пары записей, и разница в средних значениях весьма велика (более 0,77
для обоих признаков). Признаки cmp_bd, cmp_1name_c1 и cmp_bm также выглядят по- лезными: они обычно доступны в наборе данных, а разница в средних значениях совпадений и несоответствий значительна. Признаки cmp_fname_c1 и cmp_1name_c2 представляют собой нечто промежуточное: cmp_fname_c1 не очень хорошо отделяет совпадения от несовпадений (разница в средних значениях составляет всего 0,28), хотя обычно он доступен для пары запи- сей, тогда как стр_1name_c2 имеет большую разницу в средних значениях, но почти
всегда отсутствует в данных. Поэтому не совсем понятно, при каких обстоятельст-
вах мы должны включать эти признаки в нашу модель на основе текущих данных. Сейчас мы будем использовать простую модель оценки, которая ранжирует сходст- во пар записей на основе сумм значений явно хороших признаков cmp_plz, cmp_by, cmp_bd, cmp_lname_c1 и cmp_bm. Для нескольких записей, в которых отсутствуют зна- чения этих признаков, при суммировании мы будем использовать о вместо значе- ния null. Мы можем получить приблизительное представление о точности нашей простой модели, создав фрейм данных вычисленных оценок и значений столбца is_match и определив, насколько хорошо оценка модели различает совпадения и несовпадения при различных пороговых значениях.

##### Функция оценивания
Для нашей функции оценивания мы суммируем значения пяти полей (cmp_lname_cl, cmp_plz, cmp_by, cmp_bd и cmp_bm). Для этого мы будем использовать expr из pyspark.sql.functions. Функция expr преобразует строку входного выражения в столбец, который она представляет. Эта строка может даже включать несколько столбцов.

In [76]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]

In [77]:
sum_expression = " + ".join(good_features)

In [78]:
sum_expression

'cmp_lname_c1 + cmp_plz + cmp_by + cmp_bd + cmp_bm'

Функция expr принимает строку, содержащую SQL-выражение, и возвращает объект колонки, который можно использовать в методах DataFrame, таких как select, withColumn, и других.

In [80]:
from pyspark.sql.functions import expr

scored=parsed.fillna(0,subset=good_features)\
    .withColumn('score', expr(sum_expression))\
    .select('score', 'is_match')
scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



In [85]:
scored.selectExpr(f"score >= {4} as above", "is_match").show(10)

+-----+--------+
|above|is_match|
+-----+--------+
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
| true|    true|
+-----+--------+
only showing top 10 rows



In [92]:
from pyspark.sql import SparkSession, DataFrame

Метод pivot в PySpark позволяет преобразовать уникальные значения из одного столбца в заголовки столбцов и сгруппировать другие данные в соответствии с этими значениями. Этот метод полезен для создания сводных таблиц и выполнения агрегатных функций.

Аргументы

pivot_col (обязательный): Имя столбца, значения которого будут использованы в качестве заголовков новых столбцов.

values (необязательный): Список уникальных значений из столбца pivot_col, которые будут использованы для создания новых столбцов. Если не указано, PySpark автоматически определит уникальные значения.

Возвращаемое значение

Метод pivot возвращает объект GroupedData, который можно использовать для применения агрегатных функций, таких как count, sum, avg и других.

In [96]:
def crossTabs(scored: DataFrame, t: float) -> DataFrame:
    # Создаем стобец above по условию и фильтруем scored а также выбираем  столбец is_match одновременно с помощью функции selectExpr
    # Затем группируем по above
    # Затем применяем метод spark pivot и получаем столбцы из строк столбца is_match со значеняеми true или false
    # Считаем все их
    # И показываем
    return scored.selectExpr(f"score >= {t} as above", "is_match")\
        .groupBy('above')\
        .pivot(pivot_col='is_match',values=('true','false'))\
        .count()\
        .show()
    

In [97]:
crossTabs(scored, 4)
    

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+



In [98]:
crossTabs(scored, 3)

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20916| 315213|
|false|   15|5412988|
+-----+-----+-------+



In [99]:
crossTabs(scored, 2)

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20931| 596414|
|false| NULL|5131787|
+-----+-----+-------+



Несмотря на то что количество ложноположительных оценок выше, чем хотелось бы, этот более щедрый фильтр по-прежнему удаляет из нашего рассмотрения 90% несовпадающих записей, оставляя каждое положительное совпадение. Хотя это весьма неплохо, можно сделать еще лучше; подумайте, сможете ли вы найти спо- соб использовать некоторые другие значения из MatchData (как отсутствующие, так и нет), чтобы придумать функцию оценки, которая успешно идентифицирует каж- дое совпадение true ценой менее 100 ложноположительных срабатываний.